In [22]:
!pip install numpy pandas matplotlib scikit-learn sounddevice wavio librosa
!pip install tensorflow==2.12
!pip install jiwer joblib tqdm
!pip install openai-whisper
!pip install openai
!pip install pyttsx3
!pip install gtts # Added gtts installation

ERROR: Could not find a version that satisfies the requirement tensorflow==2.12 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [11]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!mkdir -p datasets/ravdess
!wget -O ravdess.zip "https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip?download=1"
!unzip ravdess.zip -d datasets/ravdess

--2025-09-10 10:21:59--  https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1188976/files/Audio_Speech_Actors_01-24.zip [following]
--2025-09-10 10:22:00--  https://zenodo.org/records/1188976/files/Audio_Speech_Actors_01-24.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 208468073 (199M) [application/octet-stream]
Saving to: ‘ravdess.zip’

ravdess.zip         100%[===================>] 198.81M  2.69MB/s    in 3m 23s  

2025-09-10 10:25:23 (1004 KB/s) - ‘ravdess.zip’ saved [208468073/208468073]

Archive:  ravdess.zip
replace datasets/ravdess/Actor_01/03-01-01-01-01-01-01.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [12]:
import os
import librosa
import numpy as np
from glob import glob
from sklearn.preprocessing import LabelEncoder
import joblib
from tqdm import tqdm

DATASET_PATH = "datasets/ravdess/"
OUTPUT_DIR = "prepared_data/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

ravdess_map = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
}

def extract_logmel(file_path, sr=16000, n_mels=64, duration=3.0):
    y, sr = librosa.load(file_path, sr=sr, duration=duration, mono=True)
    if len(y) < int(duration*sr):
        y = np.pad(y, (0, int(duration*sr)-len(y)))
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=1024, hop_length=256, n_mels=n_mels)
    logmel = librosa.power_to_db(mel)
    logmel = (logmel - np.mean(logmel)) / (np.std(logmel) + 1e-9)
    return logmel.astype(np.float32)

X, y = [], []
for fp in tqdm(glob(os.path.join(DATASET_PATH, "**", "*.wav"), recursive=True)):
    fname = os.path.basename(fp)
    parts = fname.split("-")
    if len(parts) < 3: continue
    emo_code = parts[2]
    emotion = ravdess_map.get(emo_code, None)
    if emotion is None: continue
    logmel = extract_logmel(fp)
    X.append(logmel)
    y.append(emotion)

X = np.array(X)
y = np.array(y)
print("Dataset shape:", X.shape, y.shape)

# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)
joblib.dump(le, os.path.join(OUTPUT_DIR, "label_encoder.joblib"))
np.save(os.path.join(OUTPUT_DIR, "X.npy"), X)
np.save(os.path.join(OUTPUT_DIR, "y.npy"), y_enc)

100%|██████████| 1440/1440 [00:13<00:00, 104.28it/s]


Dataset shape: (1440, 64, 188) (1440,)


In [13]:
import os
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
import librosa

DATA_DIR = "prepared_data/"
os.makedirs(DATA_DIR, exist_ok=True)

# Load the prepared data
X = np.load(os.path.join(DATA_DIR, "X.npy"))
y = np.load(os.path.join(DATA_DIR, "y.npy"))
le = joblib.load(os.path.join(DATA_DIR, "label_encoder.joblib"))

print("Loaded data shape:", X.shape, y.shape)
print("Loaded labels:", le.classes_)

# The rest of your code to build and train the model can go here.
# For example:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print("Train and test shapes:", X_train.shape, X_test.shape)

Loaded data shape: (1440, 64, 188) (1440,)
Loaded labels: ['angry' 'calm' 'disgust' 'fearful' 'happy' 'neutral' 'sad' 'surprised']


In [14]:
import numpy as np
import os
import joblib
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.model_selection import train_test_split

DATA_DIR = "prepared_data/"
MODEL_DIR = "saved_models/"
os.makedirs(MODEL_DIR, exist_ok=True)

X = np.load(os.path.join(DATA_DIR, "X.npy"))
y = np.load(os.path.join(DATA_DIR, "y.npy"))
le = joblib.load(os.path.join(DATA_DIR, "label_encoder.joblib"))

X = X[..., np.newaxis].astype('float32')
n_mels, frames = X.shape[1], X.shape[2]
num_classes = len(le.classes_)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, stratify=y)

def build_model(input_shape, num_classes):
    inp = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inp)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs=inp, outputs=out)

model = build_model((n_mels, frames, 1), num_classes)
model.compile(optimizer=optimizers.Adam(1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

ckpt = os.path.join(MODEL_DIR, "emotion_cnn_best.h5")
cb = [callbacks.ModelCheckpoint(ckpt, monitor='val_accuracy', save_best_only=True)]
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=30, batch_size=32, callbacks=cb)

model.save(os.path.join(MODEL_DIR, "emotion_cnn_final.h5"))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 64, 188, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 188, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 94, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32, 94, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 94, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 47, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16, 47, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,168 (110.03 KB)

 Trainable params: 28,168 (110.03 KB)

 Non-trainable params: 0 (0.00 B)

...

39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2749 - loss: 1.9209 - val_accuracy: 0.2963 - val_loss: 1.8965
Epoch 19/30
36/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2771 - loss: 1.8986

39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2773 - loss: 1.8970 - val_accuracy: 0.3472 - val_loss: 1.8556
Epoch 20/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.2774 - loss: 1.8706 - val_accuracy: 0.3102 - val_loss: 1.8138
Epoch 21/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.2889 - loss: 1.8463 - val_accuracy: 0.3472 - val_loss: 1.8033
Epoch 22/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3013 - loss: 1.8189 - val_accuracy: 0.3194 - val_loss: 1.7943
Epoch 23/30
36/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3249 - loss: 1.7968

39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3248 - loss: 1.7964 - val_accuracy: 0.3519 - val_loss: 1.7682
Epoch 24/30
36/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3450 - loss: 1.7633

39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3416 - loss: 1.7656 - val_accuracy: 0.3611 - val_loss: 1.8073
Epoch 25/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3133 - loss: 1.7907 - val_accuracy: 0.3148 - val_loss: 1.7640
Epoch 26/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3143 - loss: 1.7996 - val_accuracy: 0.3472 - val_loss: 1.7484
Epoch 27/30
36/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3491 - loss: 1.7367

39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3488 - loss: 1.7363 - val_accuracy: 0.3796 - val_loss: 1.7081
Epoch 28/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3413 - loss: 1.7502 - val_accuracy: 0.3194 - val_loss: 1.7480
Epoch 29/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3367 - loss: 1.7270 - val_accuracy: 0.3472 - val_loss: 1.6923
Epoch 30/30
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3271 - loss: 1.7400 - val_accuracy: 0.3704 - val_loss: 1.7269


In [15]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

model = load_model("saved_models/emotion_cnn_best.h5")

y_pred = model.predict(X_val).argmax(axis=1)
print(classification_report(y_val, y_pred, target_names=le.classes_))


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step
              precision    recall  f1-score   support

       angry       0.43      0.34      0.38        29
        calm       0.44      0.86      0.58        29
     disgust       0.50      0.34      0.41        29
     fearful       0.44      0.41      0.43        29
       happy       0.11      0.04      0.05        28
     neutral       0.00      0.00      0.00        14
         sad       0.12      0.10      0.11        29
   surprised       0.38      0.72      0.49        29

    accuracy                           0.38       216
   macro avg       0.30      0.35      0.31       216
weighted avg       0.33      0.38      0.33       216



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
!pip install pyttsx3

In [27]:
import librosa
import numpy as np
import joblib
from tensorflow.keras.models import load_model
import whisper
import google.generativeai as genai
from google.colab import userdata
import pyttsx3
from IPython.display import Audio, display
from google.colab import files # Import files here
from gtts import gTTS # Import gTTS

# Load models
emotion_model = load_model("saved_models/emotion_cnn_best.h5")
le = joblib.load("prepared_data/label_encoder.joblib")
stt_model = whisper.load_model("base")

# Securely access the API key from Colab Secrets
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print("Could not configure Gemini API. Please ensure your GOOGLE_API_KEY is set correctly in Colab Secrets. 🔑")
    raise e

gemini_model = genai.GenerativeModel("gemini-1.5-flash")

SR, DURATION, N_MELS = 16000, 3.0, 64

# The helper functions extract_logmel_file, predict_emotion, transcribe, and generate_reply are defined in cell 7086bcef.
# No need to redefine them here.

# Upload test audio file
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

# Run pipeline
text = transcribe(file_path)
emotion, conf = predict_emotion(file_path)
reply = generate_reply(text, emotion)

print("Transcript:", text)
print("Emotion:", emotion, f"(confidence {conf:.2f})")
print("AI Reply:", reply)

# --- Code to speak the reply ---
print("\nPlaying AI audio response...")

# Use gTTS for text-to-speech
tts = gTTS(text=reply, lang="en")
tts.save("ai_reply.mp3")

# Display an audio player in the Colab output
display(Audio('ai_reply.mp3', autoplay=True))
# --- End of speaking code ---

Saving gaurav test.opus to gaurav test.opus
Length of y after loading: 48000
Length of y after padding: 48000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step
Transcript:  I am going through a really hard break up and I am finding it difficult to scope. I just feel, I just feel so frustrated all the time I don't know what I do.
Emotion: calm (confidence 0.52)
AI Reply: I'm so sorry to hear you're going through such a difficult breakup.  It sounds incredibly tough, and it's completely understandable that you're feeling frustrated and overwhelmed.  It takes a lot of strength to acknowledge these feelings, even when they're hard to express.  

Breakups are incredibly challenging, and it's okay to not know what to do right now.  There's no magic solution, but allowing yourself to feel these emotions is a really important first step.  Would you like to talk more about what's specifically frustrating you, or perhaps explore some ways to cope with this difficult time?  I'm here to listen without judgme

In [ ]:
!apt-get update && apt-get install -y espeak

In [ ]:
!pip uninstall -y openai-whisper
!pip install openai-whisper

In [25]:
def extract_logmel_file(file_path, sr=SR, n_mels=N_MELS, duration=DURATION):
    y, sr = librosa.load(file_path, sr=sr, duration=duration, mono=True)
    print(f"Length of y after loading: {len(y)}") # Added print statement
    if len(y) < int(duration*sr):
        y = np.pad(y, (0, int(duration*sr)-len(y)))
    print(f"Length of y after padding: {len(y)}") # Added print statement
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=1024, hop_length=256, n_mels=n_mels)
    logmel = librosa.power_to_db(mel)
    logmel = (logmel - np.mean(logmel)) / (np.std(logmel) + 1e-9)
    return logmel.astype(np.float32)

def predict_emotion(file_path):
    logmel = extract_logmel_file(file_path)[np.newaxis, ..., np.newaxis]
    pred = emotion_model.predict(logmel)
    pred_class = np.argmax(pred, axis=1)[0]
    emotion = le.inverse_transform([pred_class])[0]
    confidence = np.max(pred)
    return emotion, confidence

def transcribe(file_path):
    result = stt_model.transcribe(file_path)
    return result["text"]

def generate_reply(text, emotion):
    prompt = f"The user said: '{text}' with a {emotion} tone. Respond in a helpful and empathetic manner, acknowledging their emotion."
    response = gemini_model.generate_content(prompt)
    return response.text